# MCP Tools Example with sik-llms

This notebook demonstrates how to use the `sik-llms` library to connect to our MCP servers and interact with tools programmatically.

## Prerequisites

Before running this notebook, ensure your `.env` file has:
- `VITE_DEV_MODE=true` (bypasses auth)
- `OPENAI_API_KEY=sk-...` (for ReasoningAgent examples)

1. **Start Docker containers** (PostgreSQL + Redis):
   ```bash
   make docker-up
   ```

2. **Run database migrations** (if needed):
   ```bash
   make migrate
   ```

3. **Start the API server** (port 8000):
   ```bash
   make run
   ```

4. **Start the Content MCP server** (port 8001) - in a separate terminal:
   ```bash
   make content-mcp-server
   ```

5. **Start the Prompt MCP server** (port 8002) - in a separate terminal:
   ```bash
   make prompt-mcp-server
   ```

6. **Node.js** (for `npx` to run `mcp-remote`):
   - Ensure Node.js is installed
   - `npx` will automatically download `mcp-remote` on first use

## Configuration

In [1]:
from dotenv import load_dotenv

# Load .env from project root
load_dotenv()

# Dummy token - works with VITE_DEV_MODE=true (API doesn't validate)
PAT_TOKEN = "bm_devtoken"

# MCP Server URLs (local development)
CONTENT_MCP_URL = "http://localhost:8001/mcp"
PROMPT_MCP_URL = "http://localhost:8002/mcp"

## Connect to MCP Servers

The `MCPClientManager` from sik-llms uses the same config format as Claude Desktop. Since our MCP servers run over HTTP, we use `mcp-remote` as a stdio-to-HTTP bridge.

In [2]:
from sik_llms.mcp_manager import MCPClientManager

# Configuration using mcp-remote to bridge stdio to HTTP
mcp_config = {
    "mcpServers": {
        "content": {
            "command": "npx",
            "args": [
                "mcp-remote",
                CONTENT_MCP_URL,
                "--header",
                f"Authorization: Bearer {PAT_TOKEN}",
            ],
        },
        "prompts": {
            "command": "npx",
            "args": [
                "mcp-remote",
                PROMPT_MCP_URL,
                "--header",
                f"Authorization: Bearer {PAT_TOKEN}",
            ],
        },
    },
}

## List Available Tools

Connect to the MCP servers and discover available tools:

In [3]:
from IPython.display import Markdown

async with MCPClientManager(mcp_config, silent=False) as mcp:
    tools = mcp.get_tools()
    md = f"Found `{len(tools)}` tools\n\n"

    for tool in tools:
        md += f"### `{tool.name}`\n\n"
        if tool.description:
            md += f"{tool.description}\n\n"
        if tool.parameters:
            md += "**Arguments:**\n\n"
            for param in tool.parameters:
                required = "(required)" if param.required else "(optional)"
                md += f"- `{param.name}` {required}"
                if param.description:
                    md += f": {param.description}"
                md += "\n"
            md += "\n"
        md += "---\n\n"

    display(Markdown(md))

Found `11` tools

### `search_bookmarks`

Search bookmarks with optional text query and tag filtering. Returns active bookmarks only.

**Arguments:**

- `query` (optional): Text to search in title, URL, description, and content
- `tags` (optional): Filter by tags
- `tag_match` (optional): Tag matching: 'all' requires ALL tags, 'any' requires ANY tag
- `sort_by` (optional): Field to sort by
- `sort_order` (optional): Sort direction
- `limit` (optional): Maximum results to return
- `offset` (optional): Number of results to skip for pagination

---

### `search_notes`

Search notes with optional text query and tag filtering. Returns active notes only.

**Arguments:**

- `query` (optional): Text to search in title, description, and content
- `tags` (optional): Filter by tags
- `tag_match` (optional): Tag matching: 'all' requires ALL tags, 'any' requires ANY tag
- `sort_by` (optional): Field to sort by
- `sort_order` (optional): Sort direction
- `limit` (optional): Maximum results to return
- `offset` (optional): Number of results to skip for pagination

---

### `search_all_content`

Search across all content types (bookmarks and notes) with unified results. Returns active content only. Each item has a 'type' field.

**Arguments:**

- `query` (optional): Text to search in title, description, URL (bookmarks), and content
- `tags` (optional): Filter by tags
- `tag_match` (optional): Tag matching: 'all' requires ALL tags, 'any' requires ANY tag
- `sort_by` (optional): Field to sort by
- `sort_order` (optional): Sort direction
- `limit` (optional): Maximum results to return
- `offset` (optional): Number of results to skip for pagination

---

### `get_content`

Get a bookmark or note by ID. Supports partial reads for large content via line range params. The 'type' field is available in search results from search_all_content.

**Arguments:**

- `id` (required): The content item ID (UUID)
- `type` (required): Content type: 'bookmark' or 'note'
- `start_line` (optional): Start line for partial read (1-indexed)
- `end_line` (optional): End line for partial read (1-indexed, inclusive)

---

### `edit_content`

Edit content using string replacement. The old_str must match exactly one location. Use search_in_content first to verify match uniqueness.

**Arguments:**

- `id` (required): The content item ID (UUID)
- `type` (required): Content type: 'bookmark' or 'note'
- `old_str` (required): Exact text to find. Include surrounding context for uniqueness.
- `new_str` (required): Replacement text. Use empty string to delete the matched text.

---

### `search_in_content`

Search within a content item's text to find matches with line numbers and context. Use before editing to verify match uniqueness and build a unique old_str.

**Arguments:**

- `id` (required): The content item ID (UUID)
- `type` (required): Content type: 'bookmark' or 'note'
- `query` (required): Text to search for (literal match)
- `fields` (optional): Fields to search (comma-separated): content, title, description
- `case_sensitive` (optional): Case-sensitive search. Default: false
- `context_lines` (optional): Lines of context before/after each match (0-10). Default: 2

---

### `create_bookmark`

Create a new bookmark.

**Arguments:**

- `url` (required): The URL to bookmark
- `title` (optional): Bookmark title
- `description` (optional): Bookmark description
- `tags` (optional): Tags to assign (lowercase with hyphens, e.g., 'machine-learning')

---

### `create_note`

Create a new note.

**Arguments:**

- `title` (required): The note title (required)
- `description` (optional): Short description/summary
- `content` (optional): Markdown content of the note
- `tags` (optional): Tags to assign (lowercase with hyphens, e.g., 'meeting-notes')

---

### `list_tags`

List all tags with their usage counts

---

### `create_prompt`

Create a new prompt template. Prompts are Jinja2 templates with defined arguments that can be used as reusable templates for AI interactions.

**Arguments:**

- `name` (required): Prompt identifier (lowercase with hyphens, e.g., 'code-review'). Must be unique for your account.
- `title` (optional): Optional human-readable display name (e.g., 'Code Review Assistant')
- `description` (optional): Optional description of what the prompt does
- `content` (required): The prompt template text (required field). Can be plain text or use Jinja2 syntax: {{ variable_name }} for placeholders, {% if var %}...{% endif %} for conditionals, {% for x in items %}...{% endfor %} for loops, {{ text|upper }} for filters. Variables must be defined in the arguments list. Undefined variables cause errors.
- `arguments` (optional): List of argument definitions for the template
- `tags` (optional): Optional tags for categorization (lowercase with hyphens)

---

### `update_prompt`

Update a prompt's content using string replacement. Optionally update arguments atomically with content changes. Use this when editing template text or when adding/removing template variables.

**Arguments:**

- `id` (required): The prompt ID (UUID)
- `old_str` (required): Exact text to find in the prompt content. Must match exactly one location. Include 3-5 lines of surrounding context for uniqueness.
- `new_str` (required): Replacement text. Use empty string to delete the matched text.
- `arguments` (optional): Optional: Replace ALL prompt arguments atomically with content update. Use this when adding/removing template variables (e.g., {{ new_var }}) to avoid validation errors. If omitted, validation uses existing arguments. If provided, this list FULLY REPLACES current arguments (not a merge).

---



## Call Tools Directly

You can call tools directly using `call_tool()`. This returns a `CallToolResult` (MCP protocol) with:

- `content`: List of content items (e.g., `TextContent` with the JSON response as text)
- `structuredContent`: Optional dict with parsed/typed data (FastMCP populates this automatically)
- `isError`: Boolean indicating if the tool call failed

In [4]:
import json

async with MCPClientManager(mcp_config) as mcp:
    # Example: List all tags
    result = await mcp.call_tool("list_tags", {})

    print("=== CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== content (list of TextContent) ===\n")
    for i, content_item in enumerate(result.content):
        print(f"[{i}] type: {content_item.type}")
        print(f"[{i}] text: {content_item.text}\n")

    print("=== structuredContent (parsed dict) ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== CallToolResult ===

isError: False

=== content (list of TextContent) ===

[0] type: text
[0] text: {"tags":[{"name":"my-list-tag","count":1}]}

=== structuredContent (parsed dict) ===

{
  "tags": [
    {
      "name": "my-list-tag",
      "count": 1
    }
  ]
}


In [18]:
async with MCPClientManager(mcp_config) as mcp:
    # Example: Search for bookmarks
    result = await mcp.call_tool("search_bookmarks", {
        "limit": 2,
    })

    print("=== CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== content (list of TextContent) ===\n")
    for i, content_item in enumerate(result.content):
        print(f"[{i}] type: {content_item.type}")
        print(f"[{i}] text: {content_item.text[:200]}..." if len(content_item.text) > 200 else f"[{i}] text: {content_item.text}")  # noqa: E501
        print()

    print("=== structuredContent (parsed dict) ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== CallToolResult ===

isError: False

=== content (list of TextContent) ===

[0] type: text
[0] text: {"items":[{"id":"019bde76-5568-735a-8411-fb5bd1b917ff","url":"https://www.google.com/?client=safari","title":"Google","description":"Search the world's information, including webpages, images, videos ...

=== structuredContent (parsed dict) ===

{
  "items": [
    {
      "id": "019bde76-5568-735a-8411-fb5bd1b917ff",
      "url": "https://www.google.com/?client=safari",
      "title": "Google",
      "description": "Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for.",
      "summary": null,
      "tags": [],
      "created_at": "2026-01-21T02:50:55.716846Z",
      "updated_at": "2026-01-21T02:50:55.716846Z",
      "last_used_at": "2026-01-21T02:50:55.716846Z",
      "deleted_at": null,
      "archived_at": null
    }
  ],
  "total": 1,
  "offset": 0,
  "limit": 2,
  "has_more"

## Create Test Data

Let's create a note that we can use to test editing:

In [ ]:
async with MCPClientManager(mcp_config) as mcp:
    # Create a test note with a deliberate typo
    result = await mcp.call_tool("create_note", {
        "title": "MCP Eval Test Note",
        "content": "This is a tset note for evaluating MCP tools.\n\nIt has a typo that we will fix using the edit_content tool.",  # noqa: E501
        "tags": ["test", "mcp-eval"],
    })
    if result.isError:
        print(f"Error: {result.content[0].text}")
    else:
        note_data = json.loads(result.content[0].text)
        note_id_original = note_data["id"]
        print(f"Created note with ID: {note_id_original}")
        print(f"Title: {note_data['title']}")
        print(f"Content: {note_data['content']}")

Created note with ID: 019bde68-32b9-7633-b93b-f0e5324bab20
Title: MCP Eval Test Note
Content: This is a tset note for evaluating MCP tools.

It has a typo that we will fix using the edit_content tool.


## Test Content Editing Tools

Now let's use the new editing tools to fix the typo:

In [7]:
# First, search for the note we created
async with MCPClientManager(mcp_config) as mcp:
    result = await mcp.call_tool("search_notes", {
        "query": "MCP Eval Test Note",
        "limit": 1,
    })

    print("=== search_notes CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== structuredContent ===\n")
    print(json.dumps(result.structuredContent, indent=2))

    # Extract note_id for subsequent cells
    if result.structuredContent.get("items"):
        note_id = result.structuredContent["items"][0]["id"]
        print(f"\nExtracted note_id: {note_id}")
    else:
        print("\nNote not found - run the create cell first")
        note_id = None

assert note_id_original == note_id, "Note ID from search does not match created note ID"

=== search_notes CallToolResult ===

isError: False

=== structuredContent ===

{
  "items": [
    {
      "id": "019bde68-32b9-7633-b93b-f0e5324bab20",
      "title": "MCP Eval Test Note",
      "description": null,
      "tags": [
        "test",
        "mcp-eval"
      ],
      "created_at": "2026-01-21T02:35:29.338253Z",
      "updated_at": "2026-01-21T02:35:29.338253Z",
      "last_used_at": "2026-01-21T02:35:29.338253Z",
      "deleted_at": null,
      "archived_at": null
    }
  ],
  "total": 1,
  "offset": 0,
  "limit": 1,
  "has_more": false
}

Extracted note_id: 019bde68-32b9-7633-b93b-f0e5324bab20


In [8]:
# Get the note content
async with MCPClientManager(mcp_config) as mcp:
    result = await mcp.call_tool("get_content", {
        "id": note_id,
        "type": "note",
    })

    print("=== get_content CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== structuredContent ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== get_content CallToolResult ===

isError: False

=== structuredContent ===

{
  "id": "019bde68-32b9-7633-b93b-f0e5324bab20",
  "title": "MCP Eval Test Note",
  "description": null,
  "tags": [
    "test",
    "mcp-eval"
  ],
  "created_at": "2026-01-21T02:35:29.338253Z",
  "updated_at": "2026-01-21T02:35:29.338253Z",
  "last_used_at": "2026-01-21T02:35:29.338253Z",
  "deleted_at": null,
  "archived_at": null,
  "content": "This is a tset note for evaluating MCP tools.\n\nIt has a typo that we will fix using the edit_content tool.",
  "content_metadata": {
    "total_lines": 3,
    "start_line": 1,
    "end_line": 3,
    "is_partial": false
  }
}


In [9]:
# Search within the content to find the typo
async with MCPClientManager(mcp_config) as mcp:
    result = await mcp.call_tool("search_in_content", {
        "id": note_id,
        "type": "note",
        "query": "tset",
    })

    print("=== search_in_content CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== structuredContent ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== search_in_content CallToolResult ===

isError: False

=== structuredContent ===

{
  "matches": [
    {
      "field": "content",
      "line": 1,
      "context": "This is a tset note for evaluating MCP tools.\n\nIt has a typo that we will fix using the edit_content tool."
    }
  ],
  "total_matches": 1
}


In [10]:
# Fix the typo using edit_content
async with MCPClientManager(mcp_config) as mcp:
    result = await mcp.call_tool("edit_content", {
        "id": note_id,
        "type": "note",
        "old_str": "tset note",
        "new_str": "test note",
    })

    print("=== edit_content CallToolResult ===\n")
    print(f"isError: {result.isError}\n")

    print("=== structuredContent ===\n")
    print(json.dumps(result.structuredContent, indent=2))

=== edit_content CallToolResult ===

isError: False

=== structuredContent ===

{
  "response_type": "minimal",
  "match_type": "exact",
  "line": 1,
  "data": {
    "id": "019bde68-32b9-7633-b93b-f0e5324bab20",
    "updated_at": "2026-01-21T02:35:57.260098Z"
  }
}


In [11]:
from sik_llms import (
    ReasoningAgent,
    user_message,
    ToolPredictionEvent,
    ToolResultEvent,
    ThinkingEvent,
    TextResponse,
)

async with MCPClientManager(mcp_config) as mcp:
    # Get tools from MCP servers
    tools = mcp.get_tools()

    # Create a reasoning agent with the MCP tools
    agent = ReasoningAgent(
        model_name="gpt-4.1-mini",
        tools=tools,
        max_iterations=10,
    )
    # Ask the agent to perform a task
    messages = [
        user_message("Find my note titled 'MCP Eval Test Note' and tell me what it's about."),
    ]

    print("Agent reasoning...\n")
    async for event in agent.stream(messages):
        if isinstance(event, ThinkingEvent):
            print(f"[Thinking] {event.content[:200]}..." if len(event.content) > 200 else f"[Thinking] {event.content}")  # noqa: E501
        elif isinstance(event, ToolPredictionEvent):
            print(f"[Tool Call] {event.name}({event.arguments})")
        elif isinstance(event, ToolResultEvent):
            result_preview = str(event.result)[:200]
            print(f"[Tool Result] {result_preview}..." if len(str(event.result)) > 200 else f"[Tool Result] {event.result}")  # noqa: E501
        elif isinstance(event, TextResponse):
            print(f"\n[Final Answer]\n{event.response}")
            print(f"\n[Tokens] Input: {event.input_tokens}, Output: {event.output_tokens}")

Agent reasoning...

[Thinking] 
[Thinking] I need to find the note titled 'MCP Eval Test Note' in the user's notes to understand what it is about.
[Tool Call] search_notes({'query': 'MCP Eval Test Note', 'limit': 1})
[Tool Result] {"items":[{"id":"019bde68-32b9-7633-b93b-f0e5324bab20","title":"MCP Eval Test Note","description":null,"tags":["test","mcp-eval"],"created_at":"2026-01-21T02:35:29.338253Z","updated_at":"2026-01-21T02...
[Thinking] I found the note titled 'MCP Eval Test Note' but it doesn't have a description. To understand what the note is about, I need to retrieve the content of this note by its ID.
[Tool Call] get_content({'id': '019bde68-32b9-7633-b93b-f0e5324bab20', 'type': 'note'})
[Tool Result] {"id":"019bde68-32b9-7633-b93b-f0e5324bab20","title":"MCP Eval Test Note","description":null,"tags":["test","mcp-eval"],"created_at":"2026-01-21T02:35:29.338253Z","updated_at":"2026-01-21T02:35:57.260...
[Thinking] I have obtained the content of the note titled 'MCP Eval Test No

## Example: Agent Updating Note Content

In [17]:
async with MCPClientManager(mcp_config) as mcp:
    tools = mcp.get_tools()

    agent = ReasoningAgent(
        model_name="gpt-4.1-mini",
        tools=tools,
        max_iterations=15,  # More iterations for multiple edits
    )

    messages = [
        user_message(
            "Find the note titled 'MCP Eval Test Note' replace any mention of tool/tools with bold formatting.",  # noqa: E501
        ),
    ]

    print("Agent working on fixing typos...\n")
    async for event in agent.stream(messages):
        if isinstance(event, ThinkingEvent):
            print(f"[Thinking] {event.content}")
        elif isinstance(event, ToolPredictionEvent):
            print(f"[Tool Call] {event.name}")
            print(f"  Arguments: {json.dumps(event.arguments, indent=4)}")
        elif isinstance(event, ToolResultEvent):
            print(f"[Tool Result] {event.name}:")
            try:
                result_data = json.loads(event.result)
                print(f"  {json.dumps(result_data, indent=4)}")
            except (json.JSONDecodeError, TypeError):
                print(f"  {event.result}")
        elif isinstance(event, TextResponse):
            print(f"\n[Final Answer]\n{event.response}")
            print(f"\n[Tokens] Input: {event.input_tokens}, Output: {event.output_tokens}")

Agent working on fixing typos...

[Thinking] 
[Thinking] I need to find the note titled 'MCP Eval Test Note' first so that I can locate the content where 'tool' or 'tools' is mentioned. After that, I will edit the note to apply bold formatting to those words wherever they appear.
[Tool Call] search_notes
  Arguments: {
    "query": "MCP Eval Test Note",
    "limit": 1
}
[Tool Result] search_notes:
  {
    "items": [
        {
            "id": "019bde68-32b9-7633-b93b-f0e5324bab20",
            "title": "MCP Eval Test Note",
            "description": null,
            "tags": [
                "test",
                "mcp-eval"
            ],
            "created_at": "2026-01-21T02:35:29.338253Z",
            "updated_at": "2026-01-21T02:45:07.189547Z",
            "last_used_at": "2026-01-21T02:48:41.529164Z",
            "deleted_at": null,
            "archived_at": null
        }
    ],
    "total": 1,
    "offset": 0,
    "limit": 1,
    "has_more": false
}
[Thinking] I have fou

## Cleanup

Delete the test notes via the API:

```bash
# Permanently delete a specific note
curl -X DELETE "http://localhost:8000/notes/{note_id}?permanent=true" \
  -H "Authorization: Bearer bm_devtoken"
```

Or filter by tag `test` or `mcp-eval` in the web UI to find and delete test notes.

In [ ]:
import httpx

async with httpx.AsyncClient() as client:
    response = await client.delete(
        f"http://localhost:8000/notes/{note_id}?permanent=true",
        headers={"Authorization": f"Bearer {PAT_TOKEN}"},
    )
    if response.status_code == 204:
        print(f"Deleted note {note_id}")
    else:
        print(f"Failed to delete: {response.status_code} - {response.text}")

---